<a href="https://colab.research.google.com/github/Samin-Sadaf7/Book-of-LLMs/blob/main/HandsOnLLM_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q accelerate>=0.27.2 peft>=0.9.0 bitsandbytes>=0.43.0 transformers>=4.38.2 trl>=0.7.11 sentencepiece>=0.1.99
!pip install -q sentence-transformers>=3.0.0 mteb>=1.1.2 datasets>=2.18.0

In [2]:
from datasets import load_dataset

# Load MNLI dataset from GLUE
# 0 = entailment, 1 = neutral, 2 = contradiction
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [3]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

In [16]:
from sentence_transformers import SentenceTransformer

# Use a base model
embedding_model = SentenceTransformer('bert-base-uncased')

In [17]:
from sentence_transformers import losses
#Define the loss function. In softmax loss, we will also need to explicitly set the number of labels
train_loss = losses.SoftmaxLoss(
    model = embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3
)

In [18]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Create an embedding similarity evaluator for stsb
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
)

In [19]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

#Define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="base_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
)

In [20]:
from sentence_transformers.trainer import SentenceTransformerTrainer

# Train embedding model
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,1.073100
200,0.952300
300,0.882800
400,0.838200
500,0.822100
600,0.824300
700,0.804000
800,0.788100
900,0.768100
1000,0.765600


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1563, training_loss=0.8090017200355262, metrics={'train_runtime': 388.8947, 'train_samples_per_second': 128.57, 'train_steps_per_second': 4.019, 'total_flos': 0.0, 'train_loss': 0.8090017200355262, 'epoch': 1.0})

In [26]:
evaluator(embedding_model)

{'pearson_cosine': 0.5974316810823376, 'spearman_cosine': 0.6583789347813761}

In [27]:
from mteb import MTEB

# Choose evaluation task
evaluation = MTEB(tasks=["Banking77Classification"])

# Calculate results
results = evaluation.run(embedding_model)
results

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- Banking77Classification, s2s

[TaskResult(task_name=Banking77Classification, scores=...)]

In [28]:
from datasets import Dataset, load_dataset

#Load MNLI dataset from
#0=entailment, 1=neutral, 2=contradiction
train_dataset = load_dataset(
    "glue",
    "mnli",
    split="train"
).select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

#(neutral/contradiction)=0 and entailment=1
mappiing={2:0, 1:0, 0:1}
train_dataset = Dataset.from_dict({
    "sentence1": train_dataset["premise"],
    "sentence2": train_dataset["hypothesis"],
    "label": [float(mappiing[label]) for label in train_dataset["label"]]
})

In [30]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Create an embedding similarity evaluator for stsb
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
)

In [31]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer

#Define model
embedding_model = SentenceTransformer("bert-base-uncased")

#Loss Function
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

#Define Training Arguments
args = SentenceTransformerTrainingArguments(
    output_dir="consineloss_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
)

#Train Model
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Step,Training Loss
100,0.232500
200,0.170600
300,0.172400
400,0.159600
500,0.152400
600,0.158400
700,0.151400
800,0.155500
900,0.148200
1000,0.146900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1563, training_loss=0.1573866608046753, metrics={'train_runtime': 349.5285, 'train_samples_per_second': 143.05, 'train_steps_per_second': 4.472, 'total_flos': 0.0, 'train_loss': 0.1573866608046753, 'epoch': 1.0})

In [32]:
#Evaluate our training model
evaluator(embedding_model)

{'pearson_cosine': 0.730035786222801, 'spearman_cosine': 0.7321006653728315}